In [60]:
!pip install jax
!pip install jaxlib

import jax
import numpy as np
import jax.numpy as jnp

from jax.numpy import linalg
from jax.numpy.linalg import multi_dot
from jax import grad, jit, vmap, pmap, random, jacfwd, jacrev
#from jax.ops import index, index_add, index_update
from jax.experimental.ode import odeint
from numpy import nanargmin, nanargmax
from numpy import sqrt

import itertools


In [104]:
N=2
def generateIndex(N):
    elements=[]
    for i in range(N+1):
        elements.append(i)
    
    return elements

all_pairs = list(itertools.product(generateIndex(N), repeat=2))
filtered_pairs = [pair for pair in all_pairs if sum(pair) <= N ]
            
def getPair(index):
    for indexx, element in enumerate(filtered_pairs):
            if indexx==index and index<len(filtered_pairs):
                return element
            elif indexx!= element and index >=len(filtered_pairs):
                error=0
                return error
            else:
                continue
    
def getIndex(pair):
    for indexx, element in enumerate(filtered_pairs):
            if element==pair:
                if sum(pair)>=0 and sum(pair)<=N:
                    return indexx
                else: 
                    error="error"
                    return error
            elif element!= pair:
                if sum(pair)>N:
                    error="error"
                    return error
                elif sum(pair)<N:
                    error="error"
                    count=0
                    for i in pair:
                        if i<0:
                            count+=1
                        else:
                            continue
                    if count>0:
                        return error
                    else:
                        continue
                

                
print(" pairs : ", filtered_pairs , "\n")


#will try and use pandas later on

 pairs :  [(0, 0), (0, 1), (0, 2), (1, 0), (1, 1), (2, 0)] 



In [105]:
def create_random_2x2_matrices(N):
    matrices = []
    for _ in range(N):
        matrix = np.random.rand(2, 2)
        matrices.append(matrix)
    return matrices

dm=[] #density matrix dm
num = len(filtered_pairs)
if num <= 0:
    print("error")
else:
    random_matrices = create_random_2x2_matrices(num)
    for i, matrix in enumerate(random_matrices, 1):
        dm.append(matrix)
        

for m in dm:
    print(m, "\n")

[[0.88947196 0.61732195]
 [0.11900822 0.18802863]] 

[[0.97084406 0.16598792]
 [0.62490641 0.08239795]] 

[[0.38222242 0.11204425]
 [0.17149599 0.52288007]] 

[[0.6283913  0.95788916]
 [0.23844823 0.28683905]] 

[[0.92365583 0.47074327]
 [0.13983909 0.8565289 ]] 

[[0.15155401 0.95479343]
 [0.03668987 0.07007976]] 



In [106]:
import pandas as pd
# Create a DataFrame with columns 'a' and 'b'
df = pd.DataFrame({'pair': filtered_pairs, 'matrix': dm})

# Convert the DataFrame to a dictionary mapping 'a' values to 'b' values
mapping_dict = df.set_index('pair')['matrix'].to_dict()

#print("Mapping Dictionary:")
print(mapping_dict)

{(0, 0): array([[0.88947196, 0.61732195],
       [0.11900822, 0.18802863]]), (0, 1): array([[0.97084406, 0.16598792],
       [0.62490641, 0.08239795]]), (0, 2): array([[0.38222242, 0.11204425],
       [0.17149599, 0.52288007]]), (1, 0): array([[0.6283913 , 0.95788916],
       [0.23844823, 0.28683905]]), (1, 1): array([[0.92365583, 0.47074327],
       [0.13983909, 0.8565289 ]]), (2, 0): array([[0.15155401, 0.95479343],
       [0.03668987, 0.07007976]])}


In [116]:
def pairfunction(pair1):
    c=getIndex(pair1)
    if c != "error":
        return mapping_dict.get(pair1)
    else:
        return np.zeros((N,N))

pairt=(0,-5)
output=pairfunction(pairt)
print(output)

def indexfunction(index1): # will make use of this soon
    pairq=getPair(index1)
    return mapping_dict.get(pairq, "Invalid Input")
  
#index=1
#output =indexfunction(index)
#print(output)

[[0. 0.]
 [0. 0.]]


In [108]:
ot=[]
num = 4
if num <= 0:
    print("error")
else:
    random_matrices = create_random_2x2_matrices(num)
    for i, matrix in enumerate(random_matrices, 1):
        ot.append(matrix)
        
H=ot[0]
p=ot[1]
V1=ot[2]
V2=ot[3]
print("H : ", H, "\n")
print("p : ", p, "\n")
print("V1 : ", V1, "\n")
print("V2 : ", V2, "\n")

H :  [[0.12148326 0.7363461 ]
 [0.9109179  0.54039509]] 

p :  [[0.070725   0.03199463]
 [0.72208042 0.67543508]] 

V1 :  [[0.78694996 0.93081617]
 [0.89555431 0.41282765]] 

V2 :  [[0.31588916 0.86080369]
 [0.41566852 0.84655046]] 



In [109]:
lamda=[1,2]
gamma=[1,2]
kb= 1.380649*(10**(-23)) #J.K^-1
temp=2.15 #K
beta=1/(kb*temp)
h_bar=1.054571817*(10**(-34))

C1=2*lamda[0]/beta*(h_bar**2)
C11=lamda[0]*gamma[0]/h_bar
C2=2*lamda[1]/beta*(h_bar**2)
C22=lamda[1]*gamma[1]/h_bar

print("lamda 1 : ", lamda[0])
print("lamda 2 : ", lamda[1])
print("gamma 1 : ", gamma[0])
print("gamma 2 : ", gamma[1])
print("kb : ", kb)
print("temp : ", temp)
print("beta : ", beta)
print("h_bar : ", h_bar)
print("C1 : ", C1)
print("C11 : ", C11)
print("C2 : ", C2)
print("C22 : ", C22)





lamda 1 :  1
lamda 2 :  2
gamma 1 :  1
gamma 2 :  2
kb :  1.380649e-23
temp :  2.15
beta :  3.368823495832521e+22
h_bar :  1.054571817e-34
C1 :  6.602433868004404e-91
C11 :  9.482521568277412e+33
C2 :  1.3204867736008808e-90
C22 :  3.7930086273109647e+34


In [110]:
for i in filtered_pairs:
    print(pairfunction(i),"\n")

[[0.88947196 0.61732195]
 [0.11900822 0.18802863]] 

[[0.97084406 0.16598792]
 [0.62490641 0.08239795]] 

[[0.38222242 0.11204425]
 [0.17149599 0.52288007]] 

[[0.6283913  0.95788916]
 [0.23844823 0.28683905]] 

[[0.92365583 0.47074327]
 [0.13983909 0.8565289 ]] 

[[0.15155401 0.95479343]
 [0.03668987 0.07007976]] 



In [111]:
for j in range(len(filtered_pairs)):
    print(indexfunction(j),"\n")

[[0.88947196 0.61732195]
 [0.11900822 0.18802863]] 

[[0.97084406 0.16598792]
 [0.62490641 0.08239795]] 

[[0.38222242 0.11204425]
 [0.17149599 0.52288007]] 

[[0.6283913  0.95788916]
 [0.23844823 0.28683905]] 

[[0.92365583 0.47074327]
 [0.13983909 0.8565289 ]] 

[[0.15155401 0.95479343]
 [0.03668987 0.07007976]] 



In [112]:
pair=(0,1)
n1=pair[0]
n2=pair[1]


In [121]:
#HEOM equation
#equation part 1
a1=-1*(1j*(jnp.dot(H,p)-jnp.dot(p,H)) + n1*gamma[0] + n2*gamma[1] )*pairfunction((n1,n2))
a2=(1j*(jnp.dot(V1,pairfunction((n1+1,n2)))- jnp.dot(pairfunction((n1+1,n2)),V1)))
a3=(1j*(jnp.dot(V2,pairfunction((n1,n2+1))) - jnp.dot(pairfunction((n1,n2+1)),V2)))
a=a1+a2+a3

#part 2
b1=(C1*(jnp.dot(V1, pairfunction((n1-1, n2))) - jnp.dot(pairfunction((n1-1, n2)), V1)))
b2=-1j*(C11*(jnp.dot(V1, pairfunction((n1-1, n2))) + jnp.dot(pairfunction((n1-1, n2)), V1)))
b=1j*(b1+b2)

#part 3
c1=(C2*(jnp.dot(V2,pairfunction((n1,n2-1))) - jnp.dot(pairfunction((n1,n2-1)),V2)))
c2=-1j*(C22*(jnp.dot(V2,pairfunction((n1,n2-1))) + jnp.dot(pairfunction((n1,n2-1)),V2)))
c=1j*(c1+c2)

#the HEOM eqn = part 1 + part 2 + part 3 = a+b+c
res=a+b+c
res.flatten().reshape(-1,1)






#a=jnp.dot(V1,pairfunction((n1+1,n2)))-jnp.dot(pairfunction((n1+1,n2)),V1)


[[3.4933348e+34-0.67826474j]
 [6.2399388e+34+0.10356808j]
 [2.2235488e+34+0.19553573j]
 [2.5693668e+34+0.23177025j]]


In [123]:
def dpdt(pair):
    n1=pair[0]
    n2=pair[1]
    
    #HEOM equation
    #equation part 1
    a1=-1*(1j*(jnp.dot(H,p)-jnp.dot(p,H)) + n1*gamma[0] + n2*gamma[1] )*pairfunction((n1,n2))
    a2=(1j*(jnp.dot(V1,pairfunction((n1+1,n2)))- jnp.dot(pairfunction((n1+1,n2)),V1)))
    a3=(1j*(jnp.dot(V2,pairfunction((n1,n2+1))) - jnp.dot(pairfunction((n1,n2+1)),V2)))
    a=a1+a2+a3
    
    #part 2
    b1=(C1*(jnp.dot(V1, pairfunction((n1-1, n2))) - jnp.dot(pairfunction((n1-1, n2)), V1)))
    b2=-1j*(C11*(jnp.dot(V1, pairfunction((n1-1, n2))) + jnp.dot(pairfunction((n1-1, n2)), V1)))
    b=1j*(b1+b2)
    
    #part 3
    c1=(C2*(jnp.dot(V2,pairfunction((n1,n2-1))) - jnp.dot(pairfunction((n1,n2-1)),V2)))
    c2=-1j*(C22*(jnp.dot(V2,pairfunction((n1,n2-1))) + jnp.dot(pairfunction((n1,n2-1)),V2)))
    c=1j*(c1+c2)
    
    #the HEOM eqn = part 1 + part 2 + part 3 = a+b+c
    res=a+b+c
    
    return res.flatten().reshape(-1,1)

print(dpdt((0,1)))

    

[[3.4933348e+34-0.67826474j]
 [6.2399388e+34+0.10356808j]
 [2.2235488e+34+0.19553573j]
 [2.5693668e+34+0.23177025j]]


In [ ]:
def solveDE():#using RK4 # not sure how implement RK4 here
    return continue

In [129]:
data1=[]
for dtpair in filtered_pairs:
    matrix=dpdt(dtpair) 
    data1.append(matrix)
    
combined_matrix_dpdt=jnp.stack(data1) # a 1xn matrix with column vector matrix elements
    
data2=[]
for pair in filtered_pairs:
    matrix=pairfunction(pair)
    data2.append(matrix)
    
combined_matrix_p=jnp.stack(data2) # a 1xn matrix with column vector matrix elements    

#dpdtmatrix=M*pmatrix
#dpdtmatrix*pmatrix^-1=M

#making dpdtmatrix and pmatrix a diagonal matrix
combined_matrix_dpdtdiagonal=jnp.diag(combined_matrix_dpdt.flatten())
combined_matrix_pdiagonal=jnp.diag(combined_matrix_p.flatten())

#solving for matrix M
pmatrixdiagonalinverse=jnp.linalg.inv(combined_matrix_pdiagonal)
M=combined_matrix_dpdtdiagonal*pmatrixdiagonalinverse
